In [1]:
import numpy as np

import os
from pathlib import Path

In [2]:
path = Path(os.getcwd())
root = Path(path.parent.parent.absolute())

preprocessed_image_path = root / 'Shared Preprocessed Objects' / 'Preprocessed Images for Inception'
model_path = root / 'Models' / 'Retrained Inception'

In [ ]:
# preprocessed_train_path = '/content/drive/MyDrive/Applications of Deep Learning - Medical Image Captioning/ROCOfull/Train/Inception_Preprocessed_Train'
# preprocessed_test_path = '/content/drive/MyDrive/Applications of Deep Learning - Medical Image Captioning/ROCOfull/Test/Inception_Preprocessed_Test'

# save_train =  '/content/drive/MyDrive/Applications of Deep Learning - Medical Image Captioning/ROCOfull/Train/Retrained_Inception_Features/'
# save_test =  '/content/drive/MyDrive/Applications of Deep Learning - Medical Image Captioning/ROCOfull/Test/Retrained_Inception_Features/'

In [4]:
import tensorflow as tf
model = tf.keras.models.load_model(model_path / 'Retrained_Inception' / 'epoch=2') # Choose best epoch
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
img (InputLayer)             [(None, 299, 299, 3)]     0         
_________________________________________________________________
model (Functional)           (None, 2048)              21802784  
_________________________________________________________________
model_1 (Functional)         (None, 745)               1526505   
Total params: 23,329,289
Trainable params: 23,294,857
Non-trainable params: 34,432
_________________________________________________________________


In [7]:
import glob
import gc

def extract_features_retrained_inception(model, path, path_save, train_test, batch_size = 32):
  files = glob.glob(str(path) + '/*')
  #print(files)

  for batch, file in enumerate(files):
    if train_test not in file:
        continue
    
    print('Extracting Features Batch = ' + str(batch))
    features = {}
    preprocessed_images = np.load(file, allow_pickle = True).item()
    imageIds = preprocessed_images.keys()
    images = np.array(list(preprocessed_images.values()))
    gc.collect()
    #print(len(imageIds), images.shape)

    extracted_features = model.predict(images, batch_size = 32, verbose = 1)

    for index, imageId in enumerate(imageIds):
      features[imageId] = extracted_features[index]
    
    #print(extracted_features.shape, features)
    
    if train_test == 'train':
        np.save(path_save / 'Train Features' / ('features_batch_' + str(batch)), features, allow_pickle = True)
    else:
        np.save(path_save / 'Test Features' / ('features_batch_' + str(batch)), features, allow_pickle = True)


In [9]:
# Extract train features using retrained inception on CUIs
extract_features_retrained_inception(model, preprocessed_image_path, model_path, 'train')
# Extract test features using retrained inception on CUIs
extract_features_retrained_inception(model, preprocessed_image_path, model_path, 'test')

Extracting Features Batch = 5
1/1 [==============================] - 0s 58ms/step
Extracting Features Batch = 6
1/1 [==============================] - 0s 55ms/step
Extracting Features Batch = 7
1/1 [==============================] - 0s 61ms/step
Extracting Features Batch = 8
1/1 [==============================] - 0s 55ms/step
Extracting Features Batch = 9
1/1 [==============================] - 0s 45ms/step
Extracting Features Batch = 10
1/1 [==============================] - 0s 35ms/step
Extracting Features Batch = 11
1/1 [==============================] - 0s 35ms/step
Extracting Features Batch = 12
1/1 [==============================] - 0s 38ms/step
Extracting Features Batch = 13
1/1 [==============================] - 0s 40ms/step
Extracting Features Batch = 14
1/1 [==============================] - 0s 38ms/step
Extracting Features Batch = 0
1/1 [==============================] - 0s 39ms/step
Extracting Features Batch = 1
1/1 [==============================] - 0s 45ms/step
Extracting 

In [18]:
# This function merges all the features batches created earlier into one single dictionary. This is possible because of the dimensionality reduction from the 
# Preprocessed images with shape (X, 299, 299, 3) into the features with shape (X, cui_vocab_size), which is 745 for the models we are running.
def Merge(dict1, dict2):
    for i in dict2.keys():
        dict1[i]=dict2[i]
    return dict1

def merge_feature_batches(path, train_test):
    if train_test == 'train':
        files = glob.glob(str(path) + '/Train Features/*')
    else:
        files = glob.glob(str(path) + '/Test Features/*')
    features_full = {}

    for file in files:
        features = np.load(file, allow_pickle = True).item()

        features_full = Merge(features_full, features)

    print(len(features_full))
    return features_full


In [20]:
features_full_train = merge_feature_batches(model_path, 'train')
np.save(model_path / 'train_features_full', features_full_train, allow_pickle=True)

features_full_test = merge_feature_batches(model_path, 'test')
np.save(model_path / 'test_features_full', features_full_test, allow_pickle=True)


100
50
